w_03 | BeautifulSoup
====

BeautifulSoup es, junto con Scrapy, el paquete más usado para scrapping web. Su uso básico tiene una lógica muy simple que consiste en proveerle el código fuente de una página (el HTML) para que construya un "árbol" con sus elementos en donde podemos buscar lo que necesitamos.

Busquemos todas las noticias de la portada de Montevideo Portal, para lo que hay que inspeccionar la fuente.

![](https://drive.google.com/uc?export=view&id=1ypYmHKOQX281hqE5aLv0J39s0-oP56A4)

In [1]:
from pprint import pprint

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.montevideo.com.uy/categoria/Noticias-310"
r = requests.get(url)
soup = BeautifulSoup(r.content)
find = soup.find_all("h2", class_="title")
results = {el.contents[0].text: el.contents[0].get("href") for el in find}
pprint(results, indent=4)

{   'Argentina registra 35.017 nuevos casos de la covid-19 y 587 decesos más': 'https://www.montevideo.com.uy/Noticias/Argentina-registra-35-017-nuevos-casos-de-la-covid-19-y-587-decesos-mas-uc788305',
    'Covid en Uruguay: 52 muertes, 555 pacientes en CTI y 3.534 nuevos casos en 23.506 análisis': 'https://www.montevideo.com.uy/Noticias/Covid-en-Uruguay-52-muertes-555-pacientes-en-CTI-y-3-534-nuevos-casos-en-23-506-analisis-uc788318',
    'El exintendente de Durazno Benjamín Irazabal será el nuevo subdirector de la OPP': 'https://www.montevideo.com.uy/Noticias/El-exintendente-de-Durazno-Benjamin-Irazabal-sera-el-nuevo-subdirector-de-la-OPP-uc788324',
    'La oposición a Netanyahu acordó y anunció la formación de un nuevo gobierno en Israel': 'https://www.montevideo.com.uy/Noticias/La-oposicion-a-Netanyahu-acordo-y-anuncio-la-formacion-de-un-nuevo-gobierno-en-Israel-uc788321',
    'Nacieron trillizas en la ciudad de Rocha y están en perfectas condiciones': 'https://www.montevideo.com.u

Hicimos lo siguiente:

1. Un request para obtener el HTML.
2. Pasamos el contenido de la respuesta a la clase ``BeautifulSoup`` (se llama igual que el paquete, pero se importa de ``bs4``), lo que devuelve un objeto que podemos procesar fácilmente.
3. Usamos el método ``find_all()`` para que encuentre todos los elementos con el tag h2 y la class "title", porque eso es lo que figura cuando hacemos inspect en un browser.
4. El texto y el link están un nivel más abajo del tag h2, por lo que tenemos que acceder a los ``contents`` (es una lista, por eso accedemos al elemento 0, el único que hay en este caso).
5. Armamos una dict comprehension que extrae el texto y el link (href) de cada elemento encontrado.

¿Por qué no buscamos directamente los tags a (que son links)? Porque necesitamos alguna forma de "filtrar" los resultados, y como ese tag no tiene demasiados detalles, recurrimos a su parent h2. Sino, pasa lo siguiente.

In [2]:
soup.find_all("a")[:10]

[<a class="icon notificaciones" href="" onclick="return false;" target="_blank" title="Notificaciones"></a>,
 <a class="later notificationsReminder" href="#">Recordar más tarde</a>,
 <a class="icon facebook" href="https://www.facebook.com/portalmvd" target="_blank" title="Facebook"></a>,
 <a class="icon twitter" href="https://twitter.com/portalmvd" target="_blank" title="Twitter"></a>,
 <a class="icon instagram" href="https://www.instagram.com/portalmvd/" target="_blank" title="Instagram"></a>,
 <a class="icon youtube" href="https://www.youtube.com/c/MontevideoPortalVideos" target="_blank" title="Youtube"></a>,
 <a class="icon spotify" href="https://open.spotify.com/show/7qCGYvcXw1WzqRH5I5UZfh" target="_blank" title="Spotify"></a>,
 <a class="icon rss" href="https://www.montevideo.com.uy/auc.aspx?27383" target="_blank" title="RSS"></a>,
 <a class="icon itunes" href="https://itunes.apple.com/app/id400466279" target="_blank" title="iTunes"></a>,
 <a class="icon android" href="https://pla

Con el método ``find_all()`` y un poco de creatividad se puede hacer prácticamente todo.

La idea ahora es extraer avisos de puestos de trabajo de El Gallito. En la URL de búsqueda hay información limitada, por lo que habrá que entrar uno por uno, para lo cual necesitamos extraer las URLs.

Vamos a concentranos en los resultados de la primera página.

El proceso es igual, inspeccionamos el elemento que nos interesa, ubicamos los tags relevantes y extraemos.

In [3]:
gallito = "https://trabajo.gallito.com.uy/buscar"
r = requests.get(gallito)
soup = BeautifulSoup(r.content)
find = soup.find_all("a", class_="post-cuadro row smB")
urls = [puesto.get("href") for puesto in find]
urls

['/anuncio/distribuidores-jvug7',
 '/anuncio/aux-de-marketing-digital-y-diseno-grafico-e545j',
 '/anuncio/cadete-b7rg3',
 '/anuncio/chofer-para-taxi-97nq4',
 '/anuncio/carnicero-cortador-zona-placido-ellauri-49273',
 '/anuncio/auxiliar-de-ventas-zq3ts',
 '/anuncio/oficiales-en-aluminio-pnpta',
 '/anuncio/chofer-6m4m6',
 '/anuncio/cadete-ura2a',
 '/anuncio/aprendiz-carpinteria-madera-5h5pg',
 '/anuncio/gana-dinero-en-donde-estes-3ag43',
 '/anuncio/chofer-de-camion-con-grua-qhpah',
 '/anuncio/cajera-administrativa-para-tienda-de-shopping-puqz6',
 '/anuncio/guardia-de-seguridad-zona-libertad-san-jose-g84hz',
 '/anuncio/operador-independiente-teletrabajo-llamados-call-mgphj',
 '/anuncio/vendedoras-es-j6jed',
 '/anuncio/promo-vendedor-de-calle-zj9zz',
 '/anuncio/vendedor-telefonico-46vhh']

La tarea ahora es constuir las URLs, hacer el request y luego el scrapping.

La ventaja de HTML es que suele estar todo en el mismo lugar. Entonces con ver la estructura en un puesto podemos saber cómo van a funcionar el resto.

In [4]:
puestos = []
with requests.Session() as s:
    for url in urls:
        full_url = f"https://trabajo.gallito.com.uy{url}"
        print(f"Procesando {full_url}")
        r = s.get(full_url)
        soup = BeautifulSoup(r.content)
        title = soup.find("div", class_="title-puesto").contents[0].text.strip()
        subtitle = soup.find("div", class_="subtitle-puesto").text.strip()
        since = soup.find("span", class_="time-text").text.strip()
        cantidad = soup.find("div", class_="span-ofertas").text.strip()
        titulo_textos = soup.find_all("div", class_="cuadro-aviso-title")
        textos = soup.find_all("div", class_="cuadro-aviso-text")
        textos = {k.text.strip(): v.text.strip() for k, v in zip(titulo_textos, textos)}
        textos.pop("Avisos similares", None)
        data_puesto = {"Descripción": title, "Empresa": subtitle, "Cantidad de ofertas": cantidad, "Fecha": since}
        data_puesto.update(textos)
        puestos.append(data_puesto)

df = pd.DataFrame(puestos)
df.head()

Procesando https://trabajo.gallito.com.uy/anuncio/distribuidores-jvug7
Procesando https://trabajo.gallito.com.uy/anuncio/aux-de-marketing-digital-y-diseno-grafico-e545j
Procesando https://trabajo.gallito.com.uy/anuncio/cadete-b7rg3
Procesando https://trabajo.gallito.com.uy/anuncio/chofer-para-taxi-97nq4
Procesando https://trabajo.gallito.com.uy/anuncio/carnicero-cortador-zona-placido-ellauri-49273
Procesando https://trabajo.gallito.com.uy/anuncio/auxiliar-de-ventas-zq3ts
Procesando https://trabajo.gallito.com.uy/anuncio/oficiales-en-aluminio-pnpta
Procesando https://trabajo.gallito.com.uy/anuncio/chofer-6m4m6
Procesando https://trabajo.gallito.com.uy/anuncio/cadete-ura2a
Procesando https://trabajo.gallito.com.uy/anuncio/aprendiz-carpinteria-madera-5h5pg
Procesando https://trabajo.gallito.com.uy/anuncio/gana-dinero-en-donde-estes-3ag43
Procesando https://trabajo.gallito.com.uy/anuncio/chofer-de-camion-con-grua-qhpah
Procesando https://trabajo.gallito.com.uy/anuncio/cajera-administrativa

,Descripción,Empresa,Cantidad de ofertas,Fecha,Responsabilidades,Funciones,Requisitos
0,DISTRIBUIDORES,GRANJA POCHA S.A SOLICITA,1 ofertas laborales,Hace 4 horas,BUSCAMOS DISTRIBUIDORES DE LECHE FRESCA PA...,,NaN
1,AUX DE MARKETING DIGITAL Y DISEÑO GRAFICO,Importante empresa,10 ofertas laborales,Hace 5 horas,Sus principales tareas serán concebir y realiz...,La búsqueda se orienta a profesionales de dise...,Experiencia de 1 año en el área de Arte - dise...
2,AUXILIAR DE DEPOSITO Y SALON,,2 ofertas laborales,Hace 7 horas,"STOCK Y FECHAS DE VENCIMIENTOS, ETC.\r\nDE LUN...",MANTENIMIENTO DEL ORDEN Y LIMPIEZA DE SALON Y ...,NaN
3,CHOFER PARA TAXI,empresa,2 ofertas laborales,Hace 9 horas,"Cuidar el auto, tener buen trato con sus compa...",Cubrir 3 días libres de dos compañeros.\r\nPre...,NaN
4,CARNICERO CORTADOR - ZONA PLACIDO ELLAURI,,1 ofertas laborales,Hace 11 horas,"ATENCION AL PUBLICO, CONOCIMIENTO DE CORTES, ...","ATENCION AL PUBLICO, CONOCIMIENTO DE CORTES, ...",Experiencia de 3 años en el área de Ventas - c...


Algunos comentarios:

1. Usamos una ``Session`` de ``requests`` para agilizar el proceso dado que estamos entrando a varias páginas del mismo sitio.
2. Para el título tenemos que revisar los ``contents``, porque es un h2 dentro de un div. No podemos buscar directamente los h2 porque hay otros títulos de ese tipo en la página.
3. El subtítulo (que tiene la información de la empresa), la cantidad de ofertas y el momento de publicación no tienen dificultades, simplemente accedemos a su ``text`` y le aplicamos el método ``strip()`` para que saque todos los espacios extra.
4. Cada puesto tiene distintos boxes con información que no están presentes en todos los avisos. Sin embargo, siempre tienen los mismos tags. Por eso  hacemos un ``find_all()`` para esos tags y luego lo procesamos con una dict comprehension.
5. Algunos avisos tienen un box de "Avisos similares" que no nos interesa por ahora, así que lo eliminamos del diccionario de textos.
6. Construimos un nuevo diccionario con el título, subtítulo, cantidad de ofertas y momento de publicación.
7. Actualizamos este diccionario con la información de los textos.
8. Agregamos el diccionario completo a la lista de ``puestos``